In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
#!uv add scipy
#!uv add matplotlib
#!uv add rich
#!uv add pvporcupine

In [4]:
import os
from rich import print

In [5]:
import pyaudio
print(pyaudio.__version__)

0.2.14

In [6]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write
import matplotlib.pyplot as plt

print(sd.query_devices())

0 bcm2835 Headphones: - (hw:0,0), ALSA (0 in, 8 out)
   1 seeed-2mic-voicecard: bcm2835-i2s-wm8960-hifi wm8960-hifi-0 (hw:2,0), ALSA (2 in, 2 out)
   2 sysdefault, ALSA (0 in, 128 out)
   3 lavrate, ALSA (0 in, 128 out)
   4 samplerate, ALSA (0 in, 128 out)
   5 speexrate, ALSA (0 in, 128 out)
   6 speex, ALSA (0 in, 1 out)
   7 upmix, ALSA (0 in, 8 out)
   8 vdownmix, ALSA (0 in, 6 out)
   9 playback, ALSA (0 in, 128 out)
  10 capture, ALSA (128 in, 0 out)
  11 dmixed, ALSA (0 in, 2 out)
  12 array, ALSA (2 in, 0 out)
  13 dmix, ALSA (0 in, 2 out)
* 14 default, ALSA (128 in, 128 out)

In [7]:
PICOVOICE_API_KEY=os.environ['PICOVOICE_API_KEY']
OPENAI_API_KEY=os.environ['OPENAI_API_KEY']

In [8]:
DEVICE = "seeed-2mic-voicecard"
audio_file = "output_3.wav"

In [ ]:
# Record 5 seconds of audio
duration = 5  
samplerate = 48000  # matches openWakeWord default
recording = sd.rec(int(duration * samplerate), 
                   samplerate=samplerate, 
                   channels=1, 
                   dtype='int16', 
                   device=DEVICE)
sd.wait()

print("Recorded shape:", recording.shape)

In [ ]:
# Play back the recorded audio
sd.play(recording, samplerate=samplerate, device=DEVICE)
sd.wait()
write(audio_file,samplerate, recording)
print(f"Saved as {audio_file}")

In [ ]:
!ls -lah

In [ ]:
## Plot audio 

In [ ]:
audio_data = recording.flatten()
time = np.arange(audio_data.shape[0]) / samplerate

In [ ]:
# Plot waveform
plt.figure(figsize=(10, 4))
plt.plot(time, audio_data, color='blue')
plt.title("Audio Waveform")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()

## RGB LEDs

In [9]:
import spidev
import time

In [10]:
spidev.__version__

'3.7'

In [83]:
spi = spidev.SpiDev()

In [84]:
spi = spidev.SpiDev()
spi.open(0, 0)
spi.max_speed_hz = 8000000

In [106]:
BRIGHTNESS_LEVELS = {
    "off": 0,
    "very-low": 5,
    "low": 5,
    "mid": 15,
    "mid-high": 20,
    "full": 31
}

COLORS = {
    "blue": [255, 0, 0],
    "light_blue": [230, 216, 173],
    "magenta": [255, 0, 255],
    "yellow": [0, 255, 255],
    "green": [0, 255, 0],
    "light_green": [144, 238, 144],
    "orange": [0, 165, 255],
    "red": [0, 0, 255]
}

In [117]:
def set_led_color(color: dict, brightness_mode: dict):
    '''
    Following APA102 SPI Protocol
    # Start Frame: [0x00, 0x00, 0x00, 0x00]
    # Format: [brightness, Blue, Green, Red]
    # Brightness: 0b111xxxxx (0–31)
    # End Frame: [0xFF, 0xFF, 0xFF, 0xFF]
    '''
    data = []
    data += [0x00, 0x00, 0x00, 0x00]
    brightness = [0b11100000 | brightness_mode]
    for _ in range(3):  # 3 LEDs on the HAT
        data += brightness + color
    data += [0xFF, 0xFF, 0xFF, 0xFF]
    spi.xfer2(data)

In [118]:
def breathe_color(color_name, cycles=3, step_delay=0.05):
    color = COLORS[color_name]
    # Smooth brightness ramp up and down
    brightness_values = list(range(0, 32, 1)) + list(range(31, -1, -1))
    for _ in range(cycles):
        for b in brightness_values:
            set_led_color(color, b)
            time.sleep(step_delay)

In [122]:
breathe_color("magenta", cycles=10, step_delay=0.03)

In [91]:
set_led_color(COLORS['magenta'], BRIGHTNESS_LEVELS['mid'])

In [94]:
set_led_color(COLORS['magenta'], BRIGHTNESS_LEVELS['full'])

In [98]:
set_led_color(COLORS['yellow'], BRIGHTNESS_LEVELS['low'])

In [93]:
set_led_color(COLORS['magenta'], BRIGHTNESS_LEVELS['off'])

In [116]:
data = []
data += [0x00, 0x00, 0x00, 0x00]
brightness = [0b11100000 | BRIGHTNESS_LEVELS['low']]
data += brightness + COLORS['red']
data += brightness + COLORS['magenta']
data += brightness + COLORS['yellow']
data += [0xFF, 0xFF, 0xFF, 0xFF]
print(f"LED Colors: {data}")
spi.xfer2(data)

LED Colors: [0, 0, 0, 0, 229, 0, 0, 255, 229, 255, 0, 255, 229, 0, 255, 255, 255, 255, 255, 255]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Wakeup word

In [ ]:
import pvporcupine
print(pvporcupine.KEYWORDS)

In [ ]:
device_index = 1  # change to your mic index from sd.query_devices()

## Check Sample Rates
for rate in [8000, 16000, 22050, 44100, 48000]:
    try:
        sd.check_input_settings(device=device_index, samplerate=rate, channels=1)
        print(f"✅ Supported: {rate} Hz")
    except Exception as e:
        print(f"❌ Not supported: {rate} Hz -> {e}")